In [1]:
import math
import numpy as np
import pandas as pd
from statsforecast.core import StatsForecast
from statsforecast.models import AutoARIMA
import pmdarima as pm
import time
import xgboost as xgb
import matplotlib as plt


from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
from tqdm.notebook import tqdm

/Users/radu/.virtualenvs/ml/lib/python3.8/site-packages/statsforecast/core.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [3]:
full_df = pd.read_csv('../data/m4/daily-train.csv')

def get_ts(full_df, index):
    df = full_df.iloc[index:index+1, 1:].transpose()
    df.columns = ['y']
    df = df[df['y'].notna()]
    return df

# df = get_ts(full_df, 6)
df = get_ts(full_df, 1)
df.tail()

,y
V1003,2978.0
V1004,2991.9
V1005,2995.3
V1006,3000.5
V1007,2968.5


In [35]:
def generate_serial_lags(feature_count):
    return list(range(1, feature_count + 1))

def get_features_from_lags(df, lag_count=7, lags=None, keep_y=False):
    """ Generates the feature (lags) that can than be used to forecast
        if the *lags* parameter is used, then the function will use the lags specified by this parameter and will
            ignore the lag_count parameter
        if lags is not used, the function will generate a sequence of features
            like [lag_1, lag_2, ..., lag_lag_count]
    """
    res = df[['y']].copy()
    
    if lags is None:
        lags = generate_serial_lags(lag_count)
    
    # The lags are computed for the *previous* value because when we forecast,
    # we cannot compute them for the current value
    res['previous'] = res['y'].shift(1)
    for lag in lags:
        res['lag_{}'.format(lag)] = res['previous'] - res['previous'].shift(lag)
    
    res['day'] = np.arange(res.shape[0]) % 7
    res['lag_to_predict'] = res['y'] - res['previous']
    
    # Ignore the first row, as it has no previous values, it cannot be predicted
    if keep_y:
        columns_to_drop = ['previous']
    else:
        columns_to_drop = ['previous', 'y']
    return res[1:].drop(columns_to_drop, axis=1)

def get_future_lags(df, feature_count=7):
#     df.iloc[-1:].shift(1, axis=1)
    res = df[['y']].copy()
    lags = ['lag_{}'.format(lag) for lag in range(1, feature_count + 1)]
    
    for i, lag in enumerate(lags):
        res[lag] = res['y'] - res['y'].shift(i + 1)
    
    res['day'] = np.arange(res.shape[0]) % 7
    
    return res

# f = get_features_from_lags(df)
f = get_features_from_lags(df,lags=[1, 2, 4, 10])
f
# f.drop('residuals', axis=1)
# get_future_lags(df)


,lag_1,lag_2,lag_4,lag_10,day,lag_to_predict
V3,NaN,NaN,NaN,NaN,1,-24.370
V4,-24.370,NaN,NaN,NaN,2,4.637
V5,4.637,-19.733,NaN,NaN,3,161.564
V6,161.564,166.201,NaN,NaN,4,96.183
V7,96.183,257.747,238.014,NaN,5,3.658
...,...,...,...,...,...,...
V1003,270.530,1761.113,1687.417,2637.019,0,-1238.472
V1004,-1238.472,-967.942,319.046,354.323,1,-221.396
V1005,-221.396,-1459.868,301.245,-6.523,2,0.163
V1006,0.163,-221.233,-1189.175,1023.760,3,1480.812


In [7]:
def train_xgb(df, in_sample_validation=False):
    xgb_reg_params = {
    #         'learning_rate':    hp.choice('learning_rate',    np.arange(0.05, 0.31, 0.1)),
        'learning_rate': 0.01,
        'max_depth':        40,
        'early_stopping_rounds': 100,
        'eval_metric': 'rmse',
        'n_estimators': 100,
    }

    if in_sample_validation:
        # Validate against in sample data
        train = df
        validation = df
    else:
        # Validate against out of sample
        train = df[:-50]
        validation = df[-50:]

    X_train = train.drop('lag_to_predict', axis=1)
    y_train = train[['lag_to_predict']]


    reg = xgb.XGBRegressor(**xgb_reg_params)
    eval_set = [(validation.drop('lag_to_predict', axis=1), validation[['lag_to_predict']])]
    reg.fit(X_train, y_train, eval_set=eval_set, verbose=False)
    
    return reg

def predict_xgb(df, model):
    return model.predict(df)

lgs = get_features_from_lags(df)
train_df = lgs[:-10]
test_df = lgs[-10:].copy()

model = train_xgb(train_df)
test_df['yhat'] = predict_xgb(test_df.drop(['lag_to_predict'], axis=1), model)
print("MSE:", mean_squared_error(test_df['lag_to_predict'], test_df['yhat']))
# # test_df

MSE: 465.10932008111195
